# _Battle of the Neighborhoods - Concert Venues_

I am a musician, drum corps alumni/fanatic and therefore like to choose my travel and vacations based upon density and location of concert venues.  I am in the process of saving money for a big vacation for when cities and venues open post-COVID-19.  The following cities are on my bucket list:

- New York, NY
- San Francisco, CA
- Seattle, WA
- Boston, MA
- Chicago, IL

I will make my selection based on a combination of density and location of concert venues.  While I understand each of the above cities host a number of events on a regular basis, I am not sure of the density of said venues nor where I should make a hotel reservation.  The problem I want to solve is to analyze concert venue locations in the above cities and find a convenient area to make a hotel reservation.

## 2. A description of the data and how it will be used to solve the problem. 

I will use the Foursquare application programming interface (API) to research and collect concert venue information for the aforementioned locations.  The data can be found here:
- Foursquare Link: https://developer.foursquare.com/docs/build-with-foursquare/categories
- Venue (concert hall) location code: 5032792091d4c4b30a586d5c
Using the Foursquare data, I will construct a map of the above cities showing each concert hall location and conduct analysis to inform a decision regarding where to go on my vacation.

A preview of the data is below.  However, before we can view the data, we need to install and import various libraries:


In [3]:
# Install geopy

!pip install geopy

In [4]:
# Install folium

!pip install folium

In [5]:
# Import other libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium

print('Libraries imported.')

Libraries imported.


In [6]:
# Establish Foursquare credentials

CLIENT_ID = 'E35M35R0TDYOGJIMFO3B5BPCFVHEZTOLIBB143AYE3MTOUVE' # your Foursquare ID
CLIENT_SECRET = 'BABTZXPQMIBOY12Y2UL15GFTOS4GWI5OCAX4VONCKFY4LKI3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: E35M35R0TDYOGJIMFO3B5BPCFVHEZTOLIBB143AYE3MTOUVE


In [7]:
# Obrain the data from Foursquare

LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Seattle, WA', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "5032792091d4c4b30a586d5c") # Concert Hall place category ID
    results[city] = requests.get(url).json()

In [8]:
# View raw data to make sure the data is correctly imported

results

{'New York, NY': {'meta': {'code': 200,
   'requestId': '5ee0ff15ad40e14f644f2e3d'},
  'response': {'suggestedFilters': {'header': 'Tap to show:',
    'filters': [{'name': 'Open now', 'key': 'openNow'},
     {'name': '$-$$$$', 'key': 'price'}]},
   'geocode': {'what': '',
    'where': 'new york ny',
    'center': {'lat': 40.742185, 'lng': -73.992602},
    'displayString': 'New York, NY, United States',
    'cc': 'US',
    'geometry': {'bounds': {'ne': {'lat': 40.882214, 'lng': -73.907},
      'sw': {'lat': 40.679548, 'lng': -74.047285}}},
    'slug': 'new-york-city-new-york',
    'longId': '72057594043056517'},
   'headerLocation': 'New York',
   'headerFullLocation': 'New York',
   'headerLocationGranularity': 'city',
   'query': 'concert hall',
   'totalResults': 80,
   'suggestedBounds': {'ne': {'lat': 40.85926316355734,
     'lng': -73.47752102706853},
    'sw': {'lat': 40.58178665371454, 'lng': -74.03809494012417}},
   'groups': [{'type': 'Recommended Places',
     'name': 'recomm

In [12]:
# Create a master dataframe to consolidate data for easy viewing and analysis.

df_master = pd.DataFrame(columns = ['City', 'Num Venues', 'MDMC'])

# City = the name of the respective city
# Num Venues = the number of concert venues in the respective city
# MDMC = mean distance from mean coordinates for the respective city cluster

df_master['City'] = cities
df_master

,City,Num Venues,MDMC
0,"New York, NY",NaN,NaN
1,"Chicago, IL",NaN,NaN
2,"San Francisco, CA",NaN,NaN
3,"Seattle, WA",NaN,NaN
4,"Boston, MA",NaN,NaN
